In [2]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_ps.csv', sep=',')

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода nunique():

In [3]:
print(melb_df['Address'].nunique())
# 13378

13378


Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.

Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:

In [4]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])
# 2/119 Railway St N
# 9/400 Dandenong Rd
# 172 Danks St

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [5]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type
print(get_street_type(melb_df['Address'].loc[177]))

St


Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

In [6]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

#Обратите внимание, что функция пишется для одного элемента столбца, а метод apply() 
#применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [7]:
#Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:
print(street_types.nunique())
# 56

56


У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:

In [8]:
display(street_types.value_counts())

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Grn             4
Fairway         4
Mews            4
Gdns            4
Righi           3
Crossway        3
Esp             2
Ridge           2
Victoria        2
Crofts          2
Athol           1
Highway         1
Cove            1
Grange          1
Res             1
Terrace         1
Qy              1
Glade           1
Nook            1
Eyrie           1
Loop            1
Dell            1
East            1
Summit          1
Grand           1
Gra             1
Hts             1
Outlook         1
Woodland        1
Ave             1
Co

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:

In [9]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [10]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [11]:
#Посмотрим на результирующее число уникальных подтипов:
print(melb_df['StreetType'].nunique())
# 11

11


Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

In [12]:
melb_df = melb_df.drop('Address', axis=1)

Таким образом, с помощью написания собственных функций и их комбинирования с методом apply() из библиотеки Pandas мы смогли извлечь информацию из признака с адресом и заменить на признак подтипа улицы. 

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1
Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

2
Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

Далее переходите к шагу 3.

3
Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

4
Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.

5
Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».



In [13]:
melb_df['Date']

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

In [14]:
#Из задачи 3.3
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(melb_df['WeekdaySale'])

0        5
1        3
2        5
3        5
4        5
        ..
13575    5
13576    5
13577    5
13578    5
13579    5
Name: WeekdaySale, Length: 13580, dtype: int32


In [15]:
#Задание 4.2
#Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. 
#Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются 
#преимущественно по выходным (суббота и воскресенье).
#Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, 
#если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
print(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 2))
        

1081198.64


In [16]:
#Задание 4.3
#Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: 
#оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
#Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', 
#больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
# заменяем имена компаний, не попавших в список популярных, на строку 'other'
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other') 

a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))

1.3


In [17]:
#Задание 4.4 (External resource)
#Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
#Функция должна возвращать опыт работы в месяцах. 
#Не забудьте привести результат к целому числу.

test_series_1 = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
])
def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)

print(test_series_1.apply(get_experience))

0    99
1    41
2    21
3     3
4    72
dtype: int64


** Дополнительные действия, что бы сохранить новый фаил а работать по уроку в 5 модуле

In [30]:
melb_df['MonthSale'] = melb_df['Date'].dt.month


In [36]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms

In [43]:
melb_df = melb_df.drop(['Landsize', 'Price', 'Lattitude', 'Longtitude'], axis=1)

In [46]:
#создаём пустой список
unique_list = []
#пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    #создаем имя столбца (имя столбца, число уникальныъ значений)
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes) #добавляем кортеж в список
    unique_list.append(item)
#создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Coulumn_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique', ignore_index = True)
#выводим её на экран
display(unique_counts)

,Coulumn_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,WeekdaySale,5,int32
3,Method,5,object
4,Regionname,8,object
5,Rooms,9,int64
6,Bathroom,9,int64
7,StreetType,11,object
8,Car,11,int64
9,MonthSale,12,int32


In [47]:
melb_df.to_csv('data/melb_data_v1.csv', index=False, sep = ',')

In [1]:
import numpy as np

In [ ]:
numpy.zeros(n, m)